In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

*Nutzungsschätzung: under aner Minute uf emm Eagle r3-Prozessor (HINWEIS: Des isch nur e Schätzung. Ihri Laufzeit ka variiere.)*

## Hintergrund {#Hintergrund}


Amplitudenverstärkung isch e universeller Quantenalgorithmus oder e Unterroutine, wo mer verwende ka, um e quadratische Beschleunigung gegenüber anere Handvoll klassische Algorithmen z'erziele. [Grovers Algorithmus](https://arxiv.org/abs/quant-ph/9605043) isch dr erschte gsi, wo diese Beschleunigung bi unstrukturierte Suchproblemen zeigt hän. D'Formulierung vo emm Groverschen Suchproblem erfordert e Orakelfunktion, wo oine oder mehreri Basiszuständ als die Zuständ markiert, wo mir finde möchte, un en Verstärkungsschaltkreis, wo d'Amplitude vo de markierte Zuständ erhöht un folglich d'verbleibende Zuständ unterdrückt.

Do zeige mir, wie mer Grover-Orakel konstruiert un den [`grover_operator()`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.grover_operator) aus dr Qiskit-Schaltkreisbibliothek verwendet, um eifach e Groversche Suchinstanz iizrichte. Des Runtime `Sampler`-Primitiv ermöglicht d'nahtlose Ausführung vo Grover-Schaltkreisen.

## Anforderungen {#Anforderungen}


Schau vor em Aafange vo diesem Tutorial, dass mir des Folgende installiert hän:

* Qiskit SDK v1.4 oder neier, mit [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)-Unterstützung
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.36 oder neier

## Setup {#Setup}

In [1]:
# Built-in modules
import math

# Imports from Qiskit
from qiskit import QuantumCircuit
from qiskit.circuit.library import grover_operator, MCMTGate, ZGate
from qiskit.visualization import plot_distribution
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Imports from Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler


def grover_oracle(marked_states):
    """Build a Grover oracle for multiple marked states

    Here we assume all input marked states have the same number of bits

    Parameters:
        marked_states (str or list): Marked states of oracle

    Returns:
        QuantumCircuit: Quantum circuit representing Grover oracle
    """
    if not isinstance(marked_states, list):
        marked_states = [marked_states]
    # Compute the number of qubits in circuit
    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Find the indices of all the '0' elements in bit-string
        zero_inds = [
            ind
            for ind in range(num_qubits)
            if rev_target.startswith("0", ind)
        ]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        # where the target bit-string has a '0' entry
        if zero_inds:
            qc.x(zero_inds)
        qc.compose(MCMTGate(ZGate(), num_qubits - 1, 1), inplace=True)
        if zero_inds:
            qc.x(zero_inds)
    return qc

## Schritt 1: Klassische Eingaben uf es Quantenproblem abbilden {#Schritt-1-Klassische-Eingaben-auf-ein-Quantenproblem-abbilden}

Grovers Algorithmus bruucht es [Orakel](/learning/courses/fundamentals-of-quantum-algorithms/grover-algorithm/introduction), wo oine oder mehreri markierte Basiszuständ spezifiziert, wobei "markiert" en Zustand mit enere Phase vo -1 bedeutet. E Controlled-Z-Gate oder sini mehrfach kontrollierte Verallgemeinerung über $N$ Qubits markiert de $2^{N}-1$-Zustand (`'1'`*$N$ Bit-String). Des Markiere vo Basiszuständen mit oim oder mehrere `'0'` in dr binäre Darstellung erfordert des Aawendem vo X-Gates uf d'entsprechende Qubits vor un nach em Controlled-Z-Gate; des entspricht enere offene Kontrolle uf diesem Qubit. Im nachfolgende Code definiere mir e Orakel, wo genau des macht un oine oder mehreri Eingabe-Basiszuständ markiert, wo durch ihri Bit-String-Darstellung definiert send. Des `MCMT`-Gate wird verwendet, um des mehrfach kontrollierte Z-Gate z'implementiere.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_brisbane'

### Specific Grover's instance

Now that we have the oracle function, we can define a specific instance of Grover search.  In this example we will mark two computational states out of the eight available in a three-qubit computational space:

In [3]:
marked_states = ["011", "100"]

oracle = grover_oracle(marked_states)
oracle.draw(output="mpl", style="iqp")

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/c150298f-0.avif" alt="Output of the previous code cell" />

In [4]:
marked_states = ["011", "100"]

oracle = grover_oracle(marked_states)
oracle.draw(output="mpl", style="iqp")

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/7baca7e2-99fc-4089-b5d8-30da56816a6a-0.avif" alt="Output of the previous code cell" />

In [5]:
marked_states = ["011", "100"]

oracle = grover_oracle(marked_states)
oracle.draw(output="mpl", style="iqp")

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/d3a26fc9-9090-4527-a749-a412661260b6-0.avif" alt="Output of the previous code cell" />

### Grover operator

The built-in Qiskit `grover_operator()` takes an oracle circuit and returns a circuit that is composed of the oracle circuit itself and a circuit that amplifies the states marked by the oracle.  Here, we use the `decompose()` method the circuit to see the gates within the operator:

In [6]:
grover_op = grover_operator(oracle)
grover_op.decompose().draw(output="mpl", style="iqp")

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/283d5265-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/grovers-algorithm/extracted-outputs/d3a26fc9-9090-4527-a749-a412661260b6-0.avif)

### Grover-Operator {#Grover-Operator}

Dr iigebaute Qiskit `grover_operator()` nimmt en Orakel-Schaltkreis entgege un gibt en Schaltkreis zruck, wo aus em Orakel-Schaltkreis selber un emm Schaltkreis besteht, wo d'vom Orakel markierte Zuständ verstärkt. Do verwende mir d'`decompose()`-Methode vo dem Schaltkreis, um d'Gates innerhalb vo em Operator z'gseh:

In [7]:
optimal_num_iterations = math.floor(
    math.pi
    / (4 * math.asin(math.sqrt(len(marked_states) / 2**grover_op.num_qubits)))
)

![Output of the previous code cell](../docs/images/tutorials/grovers-algorithm/extracted-outputs/283d5265-0.avif)

Wiederholte Aawendunge vo diesem `grover_op`-Schaltkreis verstärke d'markierte Zuständ un mache sie zu de wahrscheinlichste Bit-Strings in dr Ausgabeverteilung vo dem Schaltkreis. Es git e optimali Anzahl selle Aawendunge, wo durch des Verhältnis vo markierte Zuständen zur Gesamtzahl möglicher Berechnungszuständ bestimmt wird:

In [8]:
qc = QuantumCircuit(grover_op.num_qubits)
# Create even superposition of all basis states
qc.h(range(grover_op.num_qubits))
# Apply Grover operator the optimal number of times
qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
# Measure all qubits
qc.measure_all()
qc.draw(output="mpl", style="iqp")

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/4933ae44-0.avif" alt="Output of the previous code cell" />

### Vollständige Grover-Schaltkreis {#Vollständiger-Grover-Schaltkreis}

E vollständiges Grover-Experiment aafangt mit emm Hadamard-Gate uf jedem Qubit; des erzeugt e gleichmäßige Überlagerung aller Berechnungsbasisstände, gefolgt vom Grover-Operator (`grover_op`), wo d'optimali Anzahl vo Male widderholt wird. Do verwende mir d'`QuantumCircuit.power(INT)`-Methode, um de Grover-Operator wiederholt aazuwende.

In [9]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

circuit_isa = pm.run(qc)
circuit_isa.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/c9a3020e-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/grovers-algorithm/extracted-outputs/4933ae44-0.avif)

## Schritt 2: Problem für d'Ausführung uf Quantenhardware optimiere {#Schritt-2-Problem-für-die-Ausführung-auf-Quantenhardware-optimieren}

In [10]:
# To run on local simulator:
#   1. Use the StatevectorSampler from qiskit.primitives instead
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10_000
result = sampler.run([circuit_isa]).result()
dist = result[0].data.meas.get_counts()

![Output of the previous code cell](../docs/images/tutorials/grovers-algorithm/extracted-outputs/c9a3020e-0.avif)

## Schritt 3: Ausführe mit Qiskit-Primitiven {#Schritt-3-Ausführen-mit-Qiskit-Primitiven}

Amplitudenverstärkung isch es Sampling-Problem, wo für d'Ausführung mit em [`Sampler`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/sampler-v2)-Runtime-Primitiv geeignet isch.

Beachte, dass d'`run()`-Methode vom [Qiskit Runtime `SamplerV2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/sampler-v2) es Iterable vo `primitive unified blocks (PUBs)` akzeptiert. Für de Sampler isch jedes PUB es Iterable im Format `(circuit, parameter_values)`. Mindestens akzeptiert er aber e Lischt vo Quantenschaltkreis(en).

In [11]:
plot_distribution(dist)

<Image src="../docs/images/tutorials/grovers-algorithm/extracted-outputs/a5ef9913-0.avif" alt="Output of the previous code cell" />

## Schritt 4: Nachbearbeitung un Rückgabe vom Ergebnis im gewünschte klassische Format {#Schritt-4-Nachbearbeitung-und-Rückgabe-des-Ergebnisses-im-gewünschten-klassischen-Format}